In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch

from ariel import *

In [ ]:
def sample_normal(mean, std, T):
    return np.stack([np.random.normal(loc=mean, scale=std) for i in range(T)], axis=1)

## Validation set

In [ ]:
ids = np.arange(N)
ids_train, ids_valid = train_test_split(ids, train_size=0.8, random_state=36)
data_train = get_dataset(ids_train)
data_valid = get_dataset(ids_valid)
trainset = NoisySpectraDataset(*data_train)
validset = SpectraDataset(
    *data_valid,
    trainset.auxiliary_train_mean, trainset.auxiliary_train_std)
ids_train.shape, ids_valid.shape

In [ ]:
modelnames = ["dauntless-vortex-103", "hopeful-morning-102", "light-brook-101", "distinctive-cosmos-104", "misunderstood-glitter-105"]
modelnames = ["polished-sea-115", "noble-moon-114", "radiant-dream-112", "wandering-cosmos-113", "lilac-blaze-111"]
state_dicts = ["models/" + modelname + ".pt" for modelname in modelnames]
models = [Model(DEFAULT_HYPERPARAMETERS) for state_dict in state_dicts]
device = "cuda" if torch.cuda.is_available() else "cpu"
for model, state_dict in zip(models, state_dicts):
    model.load_state_dict(torch.load(state_dict, map_location=torch.device(device)))
len(models)

In [ ]:
outputs_valid = [model.predict(validset) for model in models]
samples_valid = np.concatenate([
    sample_normal(mean, torch.sqrt(var), T=1000) for mean, var in outputs_valid],
    axis=1)
quartiles_valid = np.quantile(samples_valid, QUARTILES, axis=1)
light_score(validset.quartiles, quartiles_valid)

In [ ]:
# 995.126070254648 (test: 975.3316644649024)
# noisy data: 989.6687695924106 (test: 975.2577801950997)

In [ ]:
regular_score(samples_valid[:500], validset.ids[:500])

In [ ]:
# 994.6852719977115 (test: 971.0450579106324)
# noisy data: 988.6791129818406 (test: 970.174913016858)

## Test set

In [ ]:
ids_test = np.arange(800)
spectra_test = read_spectra(ids_test, path="data/test/spectra.hdf5")
X_test = spectra_test[1]
auxiliary_test = read_auxiliary_table(ids_test, path="data/test/auxiliary_table.csv")
X_test = scale(X_test)
auxiliary_test = standardise(auxiliary_test, trainset.auxiliary_train_mean, trainset.auxiliary_train_std)
X_test.shape, X_test.dtype, auxiliary_test.shape, auxiliary_test.dtype

In [ ]:
with torch.no_grad():
    outputs_test = [model(X_test, auxiliary_test) for model in models]
samples_test = np.concatenate([
    sample_normal(mean, torch.sqrt(var), T=250) for mean, var in outputs_test],
    axis=1)
quartiles_test = np.quantile(samples_test, QUARTILES, axis=1)
light_track = light_track_format(quartiles_test)
light_track

In [ ]:
regular_track_format(samples_test)
samples_test.shape